Date of extracted Data: 15-05-2025 - Make sure to update scraped_date if new extracted data.

# Import required libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Loading Data

In [2]:
scraped_date = pd.Timestamp('2025-05-15')

In [3]:
joblist = pd.read_csv('jobs_raw.csv', delimiter=',')

In [4]:
joblist.shape

(275, 6)

In [5]:
joblist.head()

,title,company,category,date_posted,location,url
0,AI Automation Expert,Contra,Full Stack Programming,NEW,"San Francisco, CA",https://weworkremotely.com/company/contra
1,Remote Full-Stack TypeScript Engineer,Glama,Full Stack Programming,NEW,"Miami, FL",https://weworkremotely.com/company/glama
2,Senior React Full-stack Developer,Lemon.io,Full Stack Programming,NEW,"New York, NY",https://weworkremotely.com/company/lemon-io
3,PHP Full-Stack Developer,OnTheGoSystems,Full Stack Programming,2d,100% remote,https://weworkremotely.com/company/onthegosystems
4,Coding Tech Bootcamp - Job Guaranteed by Metana,Metana,Full Stack Programming,BOOTCAMP INFO,"No coding experience required, Land a job with...",https://metana.io/full-stack-software-engineer...


# Cleaning Data

### Renaming Columns accurately

In [6]:
joblist.rename(columns={'date_posted':'days_since_posted'}, inplace=True)

### Checking and fixing null values
Since there are only a few null values, the locations are inserted manually.

In [7]:
null_values = joblist[joblist.isna().any(axis = 1)]
print(joblist[joblist['company'].isin(null_values['company'])])

                                                title  \
59                      Linux System Engineer (M/W/D)   
83                                 Back-End Developer   
102                                   Golang Engineer   
115                            Acquisition Specialist   
160  Real Estate CRM Manager - HubSpot (ZR_22009_JOB)   

                            company                category days_since_posted  \
59   Anexia Internetdienstleistungs   Front End Programming                9d   
83                        Winna.com    Back End Programming               29d   
102                       Canonical     DevOps and Sysadmin               23d   
115                   Pie Insurance  Management and Finance               24d   
160                       BruntWork        Customer Support               27d   

    location                                                url  
59       NaN  https://weworkremotely.com/company/anexia-inte...  
83       NaN       https://weworkremot

In [8]:
joblist.loc[
    (joblist['company'] == 'Anexia Internetdienstleistungs') & (joblist['location'].isna()),'location'] = 'Austria'
joblist.loc[
    (joblist['company'] == 'Winna.com') & (joblist['location'].isna()),'location'] = 'London'
joblist.loc[
    (joblist['company'] == 'Canonical') & (joblist['location'].isna()),'location'] = 'London'
joblist.loc[
    (joblist['company'] == 'Pie Insurance') & (joblist['location'].isna()),'location'] = 'Colorado, USA'
joblist.loc[
    (joblist['company'] == 'BruntWork') & (joblist['location'].isna()),'location'] = 'Sydney, Australia'

In [9]:
print(joblist.isnull().sum())

title                0
company              0
category             0
days_since_posted    0
location             0
url                  0
dtype: int64


### Correcting inaccurate values

The unique locations are listed to check if there are any inconsistent or inaccurate values. 

In [10]:
print(joblist['location'].unique())

['San Francisco, CA' 'Miami, FL' 'New York, NY' '100% remote'
 'No coding experience required, Land a job with 1-on-1 expert support.'
 'Virtual' 'Acireale, Italy' 'New York City' 'St. Petersburg, FL' 'Texas'
 'Los Angeles' 'NYC, SF, and TLV' 'San Francisco' 'Pennsylvania'
 'New Jersey' 'Florida' 'Murmuration' 'Alabama' 'New York'
 'Sydney, Australia' 'Brooklyn, NY' 'Fully Remote' 'Rio de Janeiro'
 'Montreal' 'Remote' 'Indiana, USA' 'London' 'Germany Hamburg'
 'The Internet' 'Oakland, California, United States' 'London, UK' 'NYC'
 'London, ON Canada' 'Brussels' 'Hong Kong' 'United States' 'Spring, TX'
 'Madeira, Portugal' 'Switzerland' 'Phoenix, AZ' 'USA' 'Nashville, TN'
 'Greenwich, CT' 'Sweden' 'Australia' 'Austria' 'Ho Chi Minh, Vietnam'
 'Quebec city, Canada' 'Denver, CO' 'Tallinn, Estonia'
 'Via Sclafani, 40D, 95024 Acireale CT, Italy' 'Aarhus, Denmark'
 'Copenhagen, Denmark' 'Finland' 'Gent, Belgium' 'US Virgin Islands'
 'Novi Sad' 'Cambridge, MA' 'Stockholm' 'Scottsdale, AZ' 'Ta

In [11]:
joblist = joblist[~joblist['location'].str.contains('No coding experience required, Land a job with 1-on-1 expert support.', na=False)]

### New Column - Region based on location
With the given location data we have, the broader region is assigned to each row. This makes it easier to group the data according to location

An extra category called 'Others' is added to check which rows do not fit in the given location. Then the rows with Others are returned to check and correct their inconsistencies.

In [12]:
region_key = {
'US':['alabama','alaska','arizona','arkansas','california','colorado','connecticut',
                 'delaware','florida','georgia','hawaii','idaho','illinois','indiana','iowa',
                 'kansas','kentucky','louisiana','maine','maryland','massachusetts','michigan',
                 'minnesota','mississippi','missouri','montana','nebraska','nevada','new hampshire',
                 'new jersey','new mexico','new york','north carolina','north dakota','ohio',
                 'oklahoma','oregon','pennsylvania','rhode island','south carolina','south dakota',
                 'tennessee','texas','utah','vermont','virginia','washington','west virginia',
                 'wisconsin','wyoming','usa','united states','u.s.','us','miami','san francisco', 
                  'nyc', 'los angeles', 'brooklyn', 'st. petersburg','spring', 'phoenix', 'nashville', 
                  'greenwich', 'denver','cambridge','scottsdale','tampa','hurst','boulder','bethesda',
                  'las vegas','boston','greenwood','pittsburgh','atlanta','mclean','orlando'],
'North America ex. US':
['canada','montreal','british columbia','toronto','el salvador','panama','(north america)'],
'Europe':
['uk', 'united kingdom', 'england', 'scotland', 'wales', 'ireland', 'germany', 'france',
        'netherlands', 'sweden', 'austria', 'belgium', 'denmark', 'finland', 'estonia', 'switzerland',
        'italy', 'spain', 'portugal', 'norway', 'poland', 'czech', 'slovakia', 'hungary', 'europe', 
         'london','novi sad','gibraltar','bratislava','zevenbergen','edinburgh','andorra','stockholm'],
'Middle East':
['uae', 'united arab emirates', 'saudi', 'lebanon', 'qatar', 'kuwait', 'middle east','israel'],
'Asia':
['india', 'china', 'japan', 'vietnam', 'singapore', 'south korea', 'malaysia', 'philippines',
        'thailand', 'taiwan', 'indonesia', 'bangladesh', 'pakistan', 'sri lanka','hong kong','tokyo'],
'Australia/NZ':
['australia', 'new zealand', 'nz','melbourne'],
'South America':
['brazil', 'argentina', 'chile', 'colombia', 'peru','rio de janeiro','puerto rico'],
'Africa':
['nigeria', 'kenya', 'egypt', 'south africa', 'africa'],
'Remote': ['remote', 'virtual', 'the internet', 'anywhere']
}

def regiontype(locate):
    for region, keywords in region_key.items():
        if any(keyword in locate for keyword in keywords):
            return region
    return 'Other'

joblist['region'] = joblist['location'].str.lower().apply(regiontype)

In [13]:
print(joblist['region'].unique())

['US' 'Remote' 'Europe' 'Other' 'South America' 'North America ex. US'
 'Asia' 'Middle East' 'Australia/NZ']


In [14]:
print(joblist[joblist['region'] == 'Other']['location'])

21                    Murmuration
247    Facing History & Ourselves
274                    Magic Inc.
Name: location, dtype: object


In [15]:
joblist.loc[
(joblist['company'] == 'Murmuration'),'location'] = 'Remote'
joblist.loc[
(joblist['company'] == 'Facing History & Ourselves'),'location'] = 'Massachusetts, US'
joblist.loc[
(joblist['company'] == 'Magic Inc.'),'location'] = 'California, US'

In [16]:
region_key = {
'US':['alabama','alaska','arizona','arkansas','california','colorado','connecticut',
                 'delaware','florida','georgia','hawaii','idaho','illinois','indiana','iowa',
                 'kansas','kentucky','louisiana','maine','maryland','massachusetts','michigan',
                 'minnesota','mississippi','missouri','montana','nebraska','nevada','new hampshire',
                 'new jersey','new mexico','new york','north carolina','north dakota','ohio',
                 'oklahoma','oregon','pennsylvania','rhode island','south carolina','south dakota',
                 'tennessee','texas','utah','vermont','virginia','washington','west virginia',
                 'wisconsin','wyoming','usa','united states','u.s.','us','miami','san francisco', 
                  'nyc', 'los angeles', 'brooklyn', 'st. petersburg','spring', 'phoenix', 'nashville', 
                  'greenwich', 'denver','cambridge','scottsdale','tampa','hurst','boulder','bethesda',
                  'las vegas','boston','greenwood','pittsburgh','atlanta','mclean','orlando'],
'North America ex. US':
['canada','montreal','british columbia','toronto','el salvador','panama','(north america)'],
'Europe':
['uk', 'united kingdom', 'england', 'scotland', 'wales', 'ireland', 'germany', 'france',
        'netherlands', 'sweden', 'austria', 'belgium', 'denmark', 'finland', 'estonia', 'switzerland',
        'italy', 'spain', 'portugal', 'norway', 'poland', 'czech', 'slovakia', 'hungary', 'europe', 
         'london','novi sad','gibraltar','bratislava','zevenbergen','edinburgh','andorra','stockholm'],
'Middle East':
['israel', 'uae', 'united arab emirates', 'saudi', 'lebanon', 'qatar', 'kuwait', 'middle east'],
'Asia':
['india', 'china', 'japan', 'vietnam', 'singapore', 'south korea', 'malaysia', 'philippines',
        'thailand', 'taiwan', 'indonesia', 'bangladesh', 'pakistan', 'sri lanka','hong kong','tokyo'],
'Australia/NZ':
['australia', 'new zealand', 'nz','melbourne'],
'South America':
['brazil', 'argentina', 'chile', 'colombia', 'peru','rio de janeiro','puerto rico'],
'Africa':
['nigeria', 'kenya', 'egypt', 'south africa', 'africa'],
'Remote': ['remote', 'virtual', 'the internet', 'anywhere']
}

def regiontype(locate):
    for region, keywords in region_key.items():
        if any(keyword in locate for keyword in keywords):
            return region
    return 'Other'

joblist['region'] = joblist['location'].str.lower().apply(regiontype)

In [17]:
print(joblist['region'].unique())

['US' 'Remote' 'Europe' 'South America' 'North America ex. US' 'Asia'
 'Middle East' 'Australia/NZ']


### Fixing Date columns

The inconsistencies  in days_since_posted is fixed. Then using datetime library, a new column is created based on the days we have.

In [18]:
joblist.loc[(joblist['days_since_posted'] == 'NEW'),'days_since_posted'] = '0d'

In [19]:
print(joblist['days_since_posted'].unique())

['0d' '2d' '8d' '9d' '14d' '16d' '18d' '1d' '3d' '4d' '5d' '6d' '7d' '10d'
 '20d' '21d' '22d' '23d' '24d' '27d' '28d' '29d' '30d' '12d' '13d' '15d'
 '26d' '25d' '19d' '17d' '11d']


In [20]:
joblist['days_since_posted'] = joblist['days_since_posted'].str.rstrip('d').astype(int)

In [21]:
joblist['posted_date'] = scraped_date - pd.to_timedelta(joblist['days_since_posted'], unit='d')

In [22]:
joblist.head()

,title,company,category,days_since_posted,location,url,region,posted_date
0,AI Automation Expert,Contra,Full Stack Programming,0,"San Francisco, CA",https://weworkremotely.com/company/contra,US,2025-05-15
1,Remote Full-Stack TypeScript Engineer,Glama,Full Stack Programming,0,"Miami, FL",https://weworkremotely.com/company/glama,US,2025-05-15
2,Senior React Full-stack Developer,Lemon.io,Full Stack Programming,0,"New York, NY",https://weworkremotely.com/company/lemon-io,US,2025-05-15
3,PHP Full-Stack Developer,OnTheGoSystems,Full Stack Programming,2,100% remote,https://weworkremotely.com/company/onthegosystems,Remote,2025-05-13
5,RoR Full-Stack Developer,OnTheGoSystems,Full Stack Programming,2,100% remote,https://weworkremotely.com/company/onthegosystems,Remote,2025-05-13


In [23]:
joblist.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265 entries, 0 to 274
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   title              265 non-null    object        
 1   company            265 non-null    object        
 2   category           265 non-null    object        
 3   days_since_posted  265 non-null    int64         
 4   location           265 non-null    object        
 5   url                265 non-null    object        
 6   region             265 non-null    object        
 7   posted_date        265 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 18.6+ KB


In [24]:
joblist.to_csv('jobs_cleaned.csv',index=False)